In [ ]:
# ============ IMPORTS ============
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
import quantstats as qs

mlflow.set_experiment("triple_barrier_classification")
mlflow.sklearn.autolog()  # auto-log params, metrics, model, etc.

# ============ LOAD LABELLED DATA ============
labelled_df = pd.read_csv("data_cache/labelled_df.csv", index_col=0, parse_dates=True)

# Define X, y
target_col = "Label_7day"
drop_cols = [target_col, "Close","Barrier_Hit_Day","Near_Peak","Actual_Return_7day"]  # keep only features

feature_cols = [col for col in labelled_df.columns if col not in drop_cols]
        
X = labelled_df[feature_cols]
y = labelled_df[target_col]

# ============ SPLIT DATA ============
split_idx = int(len(labelled_df) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

In [ ]:
# =========== DEFINE MODELS ============
models = {
    "log_reg": LogisticRegression(max_iter=1000),
    "random_forest": RandomForestClassifier(n_estimators=200, max_depth=8, random_state=42),
    "svm_rbf": SVC(kernel="rbf", probability=True, random_state=42),
}

# ============ TRAIN MODELS ============
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Train model
        model.fit(X_train, y_train)
        
        # Log the trained model with signature
        signature = infer_signature(X_train, model.predict(X_train)) 
        # defines the expected input and output formats for ML model, for reliability
        model_info = mlflow.sklearn.log_model(
            model, 
            artifact_path="model",
            signature=signature
        )

In [ ]:
# ============ EVALUATE MODELS ============
results = []
for name, model in models.items():
    with mlflow.start_run(run_name=name):
        # Create evaluation dataset (X_test + targets)
        eval_data = X_test.copy()
        eval_data["Label_7day"] = y_test
        
        # Evaluate with MLflow - generates all metrics & plots automatically
        eval_result = mlflow.models.evaluate(
            model=model_info.model_uri,
            data=eval_data,
            targets="Label_7day",
            model_type="classifier",
            evaluator_config={
                "log_explainer": True,  # SHAP feature importance
                "explainer_type": "exact"
            }
        )
        
        # Collect key metrics for comparison table
        results.append({
            "Model": name,
            "Accuracy": f"{eval_result.metrics['accuracy_score']:.3f}",
            "Precision": f"{eval_result.metrics['precision_score']:.3f}",
            "Recall": f"{eval_result.metrics['recall_score']:.3f}",
            "F1-Score": f"{eval_result.metrics['f1_score']:.3f}",
            "ROC-AUC": f"{eval_result.metrics['roc_auc']:.3f}",
        })
        
# ============ DISPLAY RESULTS ============
results_df = pd.DataFrame(results)
print(results_df.to_markdown(index=False))
results_df.to_csv("data_cache/model_performance.csv", index=False)
# Also log the comparison table as an artifact in a summary run
with mlflow.start_run(run_name="model_comparison_summary"):
    results_df.to_csv("data_cache/model_performance.csv", index=False)
    mlflow.log_artifact("data_cache/model_performance.csv")

In [ ]:
    # ============ TRADING METRICS ============

# Get actual returns for train and test set
actual_returns_train = labelled_df.loc[X_train.index, "Actual_Return_7day"]
actual_returns_test = labelled_df.loc[X_test.index, "Actual_Return_7day"]

# Filter to only traded signals (where model predicted -1 or 1)
traded_mask = (y_pred != 0)
traded_returns = actual_returns_test[traded_mask]
traded_pred = y_pred[traded_mask]
traded_true = y_test[traded_mask]

# Convert to decimal returns (QuantStats expects decimals, not %)
returns_decimal = traded_returns / 100

# Calculate trading metrics
print("\n" + "="*60)
print("TRADING PERFORMANCE METRICS (QuantStats)")
print("="*60)
print(f"Win Rate:              {qs.stats.win_rate(returns_decimal):.1%}")
print(f"Profit Factor:         {qs.stats.profit_factor(returns_decimal):.2f}")
print(f"Avg Win:               {qs.stats.avg_win(returns_decimal):.2%}")
print(f"Avg Loss:              {qs.stats.avg_loss(returns_decimal):.2%}")
print(f"Max Drawdown:          {qs.stats.max_drawdown(returns_decimal):.2%}")
print(f"Sharpe Ratio:          {qs.stats.sharpe(returns_decimal):.2f}")

# Custom metric: Avg Loss When Wrong (supervisor requirement)
wrong_predictions_mask = (traded_pred != np.sign(traded_true))
wrong_returns = traded_returns[wrong_predictions_mask]
avg_loss_when_wrong = abs(wrong_returns[wrong_returns < 0].mean())

print(f"\n{'='*60}")
print(f"AVG LOSS WHEN WRONG:   {avg_loss_when_wrong:.2%}  ← SUPERVISOR METRIC")
print(f"{'='*60}")

# Calculate Expected Value manually (QuantStats doesn't have direct EV)
win_rate = qs.stats.win_rate(returns_decimal)
avg_win = qs.stats.avg_win(returns_decimal)
avg_loss = abs(qs.stats.avg_loss(returns_decimal))
expected_value = (win_rate * avg_win) - ((1 - win_rate) * avg_loss)

print(f"\nExpected Value:        {expected_value:.2%}")

# Optional: Generate HTML report for dissertation appendix
returns_series = pd.Series(returns_decimal.values, index=X_test.index[traded_mask])
qs.reports.html(returns_series, output='results/model_performance_tearsheet.html', 
                title=f'{name} Trading Performance')
print(f"\nGenerated HTML report: results/model_performance_tearsheet.html")